<a href="https://colab.research.google.com/github/BrunoPrincipi/Visualization-Queenstown-housing/blob/main/interactive_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook is part of the Visualization-Queenstown-housing project which aims to analyze data related to houses on sale in the Queenstown-Lake region and predict their prices based on a set of features extracted from the Trade Me website. The project has an educational purpose and serves as a practice and introduction to machine learning fundamentals.

# Interactive house price prediction tool for Queenstown Region based on the trained model and user Inputs

#### import the libraries and packages needed.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pickle 
import pandas as pd

In [15]:
from geopy.geocoders import Nominatim
import folium
from shapely.geometry import Polygon

In [16]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
# Create widgets for user input
Bedrooms = widgets.IntSlider(min=1, max=6, description='Bedrooms')
Bathrooms = widgets.IntSlider(min=1, max=6, description='Bathrooms')
Parking = widgets.IntSlider(min=0, max=6, description='Parking')
Floor_Area = widgets.IntSlider(min=20, max=500,step=20 ,description='Floor Area',layout=widgets.Layout(width='35%'))
Land_Area = widgets.IntSlider(min=0, max=25000, step=200, description='Land Area',layout=widgets.Layout(width='75%'))

Neighborhood_options = ['Town Centre', 'Frankton', 'Glenorchy', 'Jacks Point','Queenstown East', 'Sunshine Bay', 'Fernhill', 'Kingston','Arthurs Point',
                        'Queenstown Hill', 'Lake Hayes','Goldfield Heights', 'Lower Shotover', 'Arrowtown','Kelvin Peninsula', 'Gibbston', 'Dalefield']

neighborhood_dropdown = widgets.Dropdown(options=Neighborhood_options, description='Neighborhood')


In [18]:
# Display the widgets
# display(Bedrooms,Bathrooms,Parking,Floor_Area,Land_Area,neighborhood_dropdown)

In [19]:
# Define a function that takes user input and returns prediction
def get_prediction(Bedrooms, Bathrooms, Parking, Floor_Area, Land_Area, neighborhood_dropdown):
    
    
    # Load the scaler, encoder and model from the pickle file
    with open('/content/drive/MyDrive/Colab Notebooks/model3.pkl', 'rb') as f:
        objects_dict = pickle.load(f)
        
    # retrieve the scaler, encoder, and model objects from the dictionary
    scaler = objects_dict['scaler']
    encoder = objects_dict['oneHotEncoder']
    model_rf= objects_dict['rf_model'] 
                      
    
    # Create a dictionary of user input
    user_input = {
        'Bedrooms': Bedrooms,
        'Bathrooms': Bathrooms,
        'Parking': Parking,
        'Floor Area': Floor_Area,
        'Land Area': Land_Area,
        'Neighborhood': neighborhood_dropdown}
    
    # Convert the dictionary to a dataframe
    user_input_df = pd.DataFrame.from_dict(user_input, orient='index').T    
    
    # convert columns to float data type
    user_input_df = user_input_df.astype({'Bedrooms': float, 'Bathrooms': float, 'Parking': float,
                                          'Floor Area': float, 'Land Area': float})
    
    
    # Store in a variable the numerical features of the dataframe
    user_num = user_input_df.select_dtypes(exclude='object')
    
    # Store in a variable the categorical features of the dataframe
    user_obj = user_input_df.select_dtypes(include='object')    
    
    
    # Store in a variable the output of one hot encoding on categorical features
    user_categorical = pd.DataFrame(encoder.transform(user_obj),columns=encoder.get_feature_names_out())
    
    # Concatenate the numeric and object dataframes
    df_input_user = pd.concat([user_num,user_categorical],axis=1)
    
    # scale the input
    user_scaled = scaler.transform(df_input_user)
    
    # Prediction based on the input
    user_predict = model_rf.predict(user_scaled)
    

    return user_predict

In [20]:
# Define a function to be called on button click
def on_button_click(b):
    
    # Input values
    Bedrooms_val = Bedrooms.value
    Bathrooms_val = Bathrooms.value
    Parking_val = Parking.value
    Floor_Area_val = Floor_Area.value
    Land_Area_val = Land_Area.value
    neighborhood_val = neighborhood_dropdown.value

    # Prediction using the function get_prediction
    prediction = get_prediction(Bedrooms_val, Bathrooms_val, Parking_val, Floor_Area_val, Land_Area_val, neighborhood_val)
    
    # Formatting numpy array into NZD
    prediction_in_nzd = prediction[0]
    prediction_formatted = "${:,.2f} NZD".format(prediction_in_nzd)
    
    
    # Display the prediction
    prediction_label = widgets.Label('The predicted price is: ' + prediction_formatted)  
    display(prediction_label) 

## Select the features of the house among the options and click 'Get Prediction'

In [21]:
# Create a button and attach the function to be called on button click
button = widgets.Button(description="Get Prediction",layout=widgets.Layout(width='30%'), style={'button_color': 'orange'})
button.on_click(on_button_click)

# Display the widgets and the button
display(Bedrooms, Bathrooms, Parking, Floor_Area, Land_Area,neighborhood_dropdown,button)

IntSlider(value=1, description='Bedrooms', max=6, min=1)

IntSlider(value=1, description='Bathrooms', max=6, min=1)

IntSlider(value=0, description='Parking', max=6)

IntSlider(value=20, description='Floor Area', layout=Layout(width='35%'), max=500, min=20, step=20)

IntSlider(value=0, description='Land Area', layout=Layout(width='75%'), max=25000, step=200)

Dropdown(description='Neighborhood', options=('Town Centre', 'Frankton', 'Glenorchy', 'Jacks Point', 'Queensto…

Button(description='Get Prediction', layout=Layout(width='30%'), style=ButtonStyle(button_color='orange'))

## Creating a map to show the neighborhood / towns location

In [22]:
# Load the polygon dict
with open('/content/drive/MyDrive/Colab Notebooks/neighbo_polygons.pkl', 'rb') as f:
    # Load the dictionary from the file using pickle.load()
    neighbo_polygons = pickle.load(f)

In [23]:
# Create a folium map to show the neignobhood location
# Map centered on the polygons
m = folium.Map(location=[-45.03372096 , 168.6725867 ], zoom_start=10, width=1400, height=600)
    

for key, value in neighbo_polygons.items():
    poly = neighbo_polygons[key]
    name = key    
    
    # Create a polygon using Shapely
    polygon = Polygon(poly)

    # Add the polygon to the map
    folium.Polygon(locations=polygon.exterior.coords, color='blue', fill=True, fill_color='blue', popup=name).add_to(m)

# Display the map
display(m)